In [2]:
!pip install pennylane

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 957 kB 15.3 MB/s 
     |████████████████████████████████| 9.3 MB 25.9 MB/s 
     |████████████████████████████████| 1.6 MB 53.5 MB/s 
     |████████████████████████████████| 108 kB 71.3 MB/s 


In [3]:
import pandas as pd
import numpy as np
from pennylane import numpy as np
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler

import pennylane as qml
from pennylane.templates.embeddings import AngleEmbedding, AmplitudeEmbedding
from pennylane.optimize import AdamOptimizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


import time
start = time.time()

In [4]:
df = pd.read_csv('/content/ClaMP_Integrated-5184.csv', sep=',')

In [5]:
df.drop(['packer_type'],axis=1,inplace=True)

In [6]:
df

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,256,4,1,0,1,...,1,1,3,0,6.603616,5.443362,1181520,6.627552,1,0
1,144,3,4,65535,184,184,4,1,0,1,...,1,1,3,0,5.205926,2.123522,7680,5.318221,0,0
2,144,3,4,65535,184,272,5,1,0,1,...,1,1,4,0,6.238000,3.380859,57872,6.507758,1,0
3,144,3,4,65535,184,184,1,1,0,1,...,1,0,1,0,0.000000,0.000000,95616,4.575092,1,0
4,144,3,4,65535,184,224,5,1,0,1,...,1,1,4,0,6.355626,0.702621,48128,5.545531,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5205,144,3,4,65535,184,216,5,1,1,1,...,1,0,5,0,6.174602,3.155928,365568,7.546568,0,1
5206,80,2,4,65535,184,256,7,1,0,1,...,1,4,3,0,0.000000,0.000000,98816,6.947195,0,1
5207,144,3,4,65535,184,216,5,1,0,1,...,1,2,3,0,6.503422,3.790871,227328,7.823114,0,1
5208,144,3,4,65535,184,248,5,1,1,1,...,1,1,4,0,6.115208,7.919091,271616,7.886012,0,1


In [7]:
from sklearn.utils import shuffle
df = shuffle(df)

In [8]:
df

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
1367,144,3,4,65535,184,176,1,1,0,1,...,1,0,1,0,0.000000,0.000000,2048,3.453674,1,0
5148,144,3,4,65535,184,192,2,1,1,1,...,1,0,2,0,3.241083,0.000000,12416,3.277153,0,1
4646,144,3,4,65535,184,248,3,1,1,1,...,1,2,1,0,0.000000,6.439252,111104,5.771888,0,1
3624,144,3,4,65535,184,184,3,1,1,1,...,1,0,3,0,4.697919,0.000000,495616,7.422419,0,1
2637,144,3,4,65535,184,128,3,1,0,1,...,1,1,2,1,5.622062,0.000000,28672,5.534846,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3345,144,3,4,65535,184,240,4,1,1,1,...,1,1,3,0,6.293836,7.988200,149504,6.825461,0,1
4258,144,3,4,65535,184,224,4,1,0,1,...,1,1,3,1,5.272288,7.591829,40960,5.935469,0,1
4020,144,3,4,65535,184,240,4,1,1,1,...,1,2,2,0,3.465415,1.282154,102400,5.700889,0,1
66,144,3,4,65535,184,232,5,1,0,1,...,1,1,4,0,6.573675,2.437202,82944,6.369040,0,0


In [9]:
# Separate class
class_0 = df[df['class'] == 0]
class_1 = df[df['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (2488, 69)
class 1: (2722, 69)


In [10]:
df = df.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')

In [11]:
df

,index,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,1367,144,3,4,65535,184,176,1,1,0,...,1,0,1,0,0.000000,0.000000,2048,3.453674,1,0
1,5148,144,3,4,65535,184,192,2,1,1,...,1,0,2,0,3.241083,0.000000,12416,3.277153,0,1
2,4646,144,3,4,65535,184,248,3,1,1,...,1,2,1,0,0.000000,6.439252,111104,5.771888,0,1
3,3624,144,3,4,65535,184,184,3,1,1,...,1,0,3,0,4.697919,0.000000,495616,7.422419,0,1
4,2637,144,3,4,65535,184,128,3,1,0,...,1,1,2,1,5.622062,0.000000,28672,5.534846,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5205,3345,144,3,4,65535,184,240,4,1,1,...,1,1,3,0,6.293836,7.988200,149504,6.825461,0,1
5206,4258,144,3,4,65535,184,224,4,1,0,...,1,1,3,1,5.272288,7.591829,40960,5.935469,0,1
5207,4020,144,3,4,65535,184,240,4,1,1,...,1,2,2,0,3.465415,1.282154,102400,5.700889,0,1
5208,66,144,3,4,65535,184,232,5,1,0,...,1,1,4,0,6.573675,2.437202,82944,6.369040,0,0


In [12]:
df.drop(['index'],axis=1,inplace=True)

In [13]:
df

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,176,1,1,0,1,...,1,0,1,0,0.000000,0.000000,2048,3.453674,1,0
1,144,3,4,65535,184,192,2,1,1,1,...,1,0,2,0,3.241083,0.000000,12416,3.277153,0,1
2,144,3,4,65535,184,248,3,1,1,1,...,1,2,1,0,0.000000,6.439252,111104,5.771888,0,1
3,144,3,4,65535,184,184,3,1,1,1,...,1,0,3,0,4.697919,0.000000,495616,7.422419,0,1
4,144,3,4,65535,184,128,3,1,0,1,...,1,1,2,1,5.622062,0.000000,28672,5.534846,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5205,144,3,4,65535,184,240,4,1,1,1,...,1,1,3,0,6.293836,7.988200,149504,6.825461,0,1
5206,144,3,4,65535,184,224,4,1,0,1,...,1,1,3,1,5.272288,7.591829,40960,5.935469,0,1
5207,144,3,4,65535,184,240,4,1,1,1,...,1,2,2,0,3.465415,1.282154,102400,5.700889,0,1
5208,144,3,4,65535,184,232,5,1,0,1,...,1,1,4,0,6.573675,2.437202,82944,6.369040,0,0


In [14]:
df_5percent=df[0:1042] 


In [15]:
df_5percent

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,176,1,1,0,1,...,1,0,1,0,0.000000,0.000000,2048,3.453674,1,0
1,144,3,4,65535,184,192,2,1,1,1,...,1,0,2,0,3.241083,0.000000,12416,3.277153,0,1
2,144,3,4,65535,184,248,3,1,1,1,...,1,2,1,0,0.000000,6.439252,111104,5.771888,0,1
3,144,3,4,65535,184,184,3,1,1,1,...,1,0,3,0,4.697919,0.000000,495616,7.422419,0,1
4,144,3,4,65535,184,128,3,1,0,1,...,1,1,2,1,5.622062,0.000000,28672,5.534846,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1037,144,3,4,65535,184,240,4,1,1,1,...,1,0,4,0,7.427548,7.327010,114688,7.112183,1,1
1038,144,3,4,65535,184,128,3,1,0,1,...,1,1,2,1,4.944369,0.000000,36864,3.826180,1,0
1039,80,2,4,65535,184,256,7,1,0,1,...,1,4,3,0,0.000000,0.000000,79872,6.253514,0,1
1040,144,3,4,65535,184,232,5,1,0,1,...,1,1,4,0,5.576599,0.162990,6656,4.145842,1,0


In [16]:
# Separate class
class_0 = df_5percent[df_5percent['class'] == 0]
class_1 = df_5percent[df_5percent['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (509, 69)
class 1: (533, 69)


In [17]:
# Read out CSV and sets/samples creation

df_5percent = df_5percent.astype(float)
train,test = train_test_split(df_5percent, test_size=0.30, random_state=2)
train_set = train
test_set = test
train_set = train_set.sample(160)
test_set = test_set.sample(40)
np.random.seed(42)

In [18]:
df_5percent.info

<bound method DataFrame.info of       e_cblp  e_cp  e_cparhdr  e_maxalloc   e_sp  e_lfanew  NumberOfSections  \
0      144.0   3.0        4.0     65535.0  184.0     176.0               1.0   
1      144.0   3.0        4.0     65535.0  184.0     192.0               2.0   
2      144.0   3.0        4.0     65535.0  184.0     248.0               3.0   
3      144.0   3.0        4.0     65535.0  184.0     184.0               3.0   
4      144.0   3.0        4.0     65535.0  184.0     128.0               3.0   
...      ...   ...        ...         ...    ...       ...               ...   
1037   144.0   3.0        4.0     65535.0  184.0     240.0               4.0   
1038   144.0   3.0        4.0     65535.0  184.0     128.0               3.0   
1039    80.0   2.0        4.0     65535.0  184.0     256.0               7.0   
1040   144.0   3.0        4.0     65535.0  184.0     232.0               5.0   
1041   144.0   3.0        4.0     65535.0  184.0     256.0               3.0   

      C

In [19]:
df_5percent.describe()

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
count,1042.000000,1042.000000,1042.000000,1042.000000,1042.000000,1042.000000,1042.000000,1042.000000,1042.000000,1042.0,...,1042.000000,1042.000000,1042.000000,1042.000000,1042.000000,1042.000000,1.042000e+03,1042.000000,1042.000000,1042.000000
mean,138.188100,2.902111,3.971209,65065.909789,183.631478,223.074856,4.674664,0.988484,0.338772,1.0,...,0.999040,1.345489,3.329175,0.153551,5.041429,2.618863,6.665602e+05,6.387864,0.542226,0.511516
std,20.435488,0.358800,0.332568,5377.907736,20.212571,49.201620,1.915785,0.106746,0.473519,0.0,...,0.030979,1.577223,1.148901,0.360691,2.471371,2.705468,3.385897e+06,1.111116,0.498453,0.500107
min,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,1.000000,0.000000,0.000000,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.536000e+03,0.947785,0.000000,0.000000
25%,144.000000,3.000000,4.000000,65535.000000,184.000000,208.000000,4.000000,1.000000,0.000000,1.0,...,1.000000,1.000000,3.000000,0.000000,4.977259,0.000000,6.641600e+04,5.700867,0.000000,0.000000
50%,144.000000,3.000000,4.000000,65535.000000,184.000000,232.000000,5.000000,1.000000,0.000000,1.0,...,1.000000,1.000000,4.000000,0.000000,6.184643,1.690023,1.207200e+05,6.390918,1.000000,1.000000
75%,144.000000,3.000000,4.000000,65535.000000,184.000000,248.000000,5.000000,1.000000,1.000000,1.0,...,1.000000,2.000000,4.000000,0.000000,6.524017,4.826927,2.947145e+05,7.345440,1.000000,1.000000
max,144.000000,3.000000,4.000000,65535.000000,512.000000,512.000000,29.000000,1.000000,1.000000,1.0,...,1.000000,26.000000,7.000000,1.000000,7.998838,7.999620,8.631331e+07,7.999864,1.000000,1.000000


In [20]:
# Separation of labels

x_train = train_set
y_train = train_set[['class']]

x_test = test_set
y_test = test_set[['class']]

In [21]:
# Reduce dimensions using PCA to fit the dimensions with the qubits

n_dim = 2
pca = PCA(n_components=n_dim)
pca.fit(x_train)

x_train = pca.transform(x_train)

pca.fit(x_test)
x_test = pca.transform(x_test)

In [22]:
# Normalize

std_scale = StandardScaler().fit(x_train)
data = std_scale.transform(x_train)

std_scale = StandardScaler().fit(x_test)
x_test = std_scale.transform(x_test)

In [23]:
# Review the balance of the target variable in train

y_train.value_counts(normalize=True)*100

class
1.0      51.25
0.0      48.75
dtype: float64

In [24]:
# Review the balance of the target variable in test

y_test.value_counts(normalize=True)*100

class
1.0      52.5
0.0      47.5
dtype: float64

In [25]:
# Angle Encoding

num_qubits = n_dim

dev = qml.device('default.qubit', wires = num_qubits)

@qml.qnode(dev)
def circuit(parameters, data):
    for i in range(num_qubits):
        qml.Hadamard(wires = i)
    
    AngleEmbedding(features = data, wires = range(num_qubits), rotation = 'Y')
    
    qml.StronglyEntanglingLayers(weights = parameters, wires = range(num_qubits))
    
    return qml.expval(qml.PauliZ(0))

In [26]:
num_layers = 5
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

print(weights_init, bias_init)

[[[ 0.00496714 -0.00138264  0.00647689]
  [ 0.0152303  -0.00234153 -0.00234137]]

 [[ 0.01579213  0.00767435 -0.00469474]
  [ 0.0054256  -0.00463418 -0.0046573 ]]

 [[ 0.00241962 -0.0191328  -0.01724918]
  [-0.00562288 -0.01012831  0.00314247]]

 [[-0.00908024 -0.01412304  0.01465649]
  [-0.00225776  0.00067528 -0.01424748]]

 [[-0.00544383  0.00110923 -0.01150994]
  [ 0.00375698 -0.00600639 -0.00291694]]] 0.0


In [27]:
circuit(weights_init, data[0])

tensor(-0.03748883, requires_grad=True)

In [28]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias


In [29]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss


In [30]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [31]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [32]:
Y = np.array(y_train.values[:,0] * 2 - np.ones(len(y_train.values[:,0])), requires_grad = False)  # shift label from {0, 1} to {-1, 1}
X = np.array(data, requires_grad=False)

for i in range(5):
    print("X = {}, Y = {: d}".format(list(X[i]), int(Y[i])))

X = [tensor(3.7742647, requires_grad=False), tensor(0.08427472, requires_grad=False)], Y =  1
X = [tensor(-0.22618685, requires_grad=False), tensor(0.95509163, requires_grad=False)], Y = -1
X = [tensor(-0.18639731, requires_grad=False), tensor(-0.22587622, requires_grad=False)], Y =  1
X = [tensor(-0.18578968, requires_grad=False), tensor(-0.25486007, requires_grad=False)], Y = -1
X = [tensor(-0.25251401, requires_grad=False), tensor(2.03205917, requires_grad=False)], Y = -1


In [33]:
opt = AdamOptimizer(stepsize=0.1, beta1=0.9, beta2=0.99, eps=1e-08)
batch_size = 10

In [34]:
%%time
weights = weights_init
bias = bias_init

wbest = 0
bbest = 0
abest = 0

for it in range(100):

    # weights update by one optimizer step

    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute the accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]
    
    if accuracy(Y, predictions) > abest:
        wbest = weights
        bbest = bias
        abest = accuracy(Y, predictions)
        print('New best')

    acc = accuracy(Y, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), acc
        )
    )

New best
Iter:     1 | Cost: 1.0934727 | Accuracy: 0.5000000 
New best
Iter:     2 | Cost: 0.9761850 | Accuracy: 0.5125000 
Iter:     3 | Cost: 1.0337812 | Accuracy: 0.5062500 
Iter:     4 | Cost: 1.0063586 | Accuracy: 0.5062500 
Iter:     5 | Cost: 0.9675270 | Accuracy: 0.5062500 
New best
Iter:     6 | Cost: 0.9547824 | Accuracy: 0.6625000 
Iter:     7 | Cost: 0.9415115 | Accuracy: 0.6437500 
Iter:     8 | Cost: 0.9325957 | Accuracy: 0.6187500 
Iter:     9 | Cost: 0.9201403 | Accuracy: 0.6562500 
Iter:    10 | Cost: 0.9448686 | Accuracy: 0.5187500 
Iter:    11 | Cost: 1.0079841 | Accuracy: 0.5187500 
Iter:    12 | Cost: 1.0183297 | Accuracy: 0.5187500 
Iter:    13 | Cost: 0.9856939 | Accuracy: 0.5125000 
Iter:    14 | Cost: 0.9362686 | Accuracy: 0.5125000 
Iter:    15 | Cost: 0.9111385 | Accuracy: 0.6000000 
Iter:    16 | Cost: 0.9242179 | Accuracy: 0.6000000 
Iter:    17 | Cost: 0.9804800 | Accuracy: 0.5625000 
Iter:    18 | Cost: 0.9300077 | Accuracy: 0.5875000 
Iter:    19 | Cost:

In [35]:
Yte = np.array(y_test.values[:,0] * 2 - np.ones(len(y_test.values[:,0])), requires_grad = False)
Xte = np.array(normalize(x_test), requires_grad=False)

In [36]:
predictions = [np.sign(variational_classifier(wbest, bbest, x)) for x in Xte]
pred = [np.sign(variational_classifier(wbest, bbest, x)) for x in X]
acc = accuracy(Yte, predictions)

print(f'Cost: {cost(wbest, bbest, Xte, Yte)}, Accuracy: {np.round(acc, 2) * 100}%')

Cost: 1.3493167403319895, Accuracy: 28.000000000000004%


In [37]:
pd.DataFrame((predictions, Yte), ('Predictions', 'Test')).T

,Predictions,Test
0,-1.0,1.0
1,1.0,1.0
2,1.0,1.0
3,1.0,-1.0
4,-1.0,1.0
5,-1.0,-1.0
6,1.0,-1.0
7,-1.0,1.0
8,1.0,-1.0
9,-1.0,1.0


In [38]:
# Print the classification report and important metrics

print(metrics.classification_report(predictions,Yte))
print(metrics.precision_score(predictions,Yte))
print(metrics.recall_score(predictions,Yte))
print(metrics.f1_score(predictions,Yte))
print(metrics.balanced_accuracy_score(predictions,Yte))

              precision    recall  f1-score   support

        -1.0       0.37      0.29      0.33        24
         1.0       0.19      0.25      0.22        16

    accuracy                           0.28        40
   macro avg       0.28      0.27      0.27        40
weighted avg       0.30      0.28      0.28        40

0.19047619047619047
0.25
0.2162162162162162
0.27083333333333337
